# Motion Chart 01 - Data Preparation

Motion Bubble Chart Data Preparation

<img src="http://wiki.stat.ucla.edu/socr/uploads/b/ba/SOCR_Activities_MotionCharts_HPI_070109_Fig6_9_Animation.gif" style="float:left">


#### 이승준 http://fb.com/plusjune

# 1995~2015 시가총액
https://gist.githubusercontent.com/plusjune/d5c30a3377d43cd53884
    

In [2]:
%matplotlib inline

import pandas as pd
import matplotlib.pylab as plt
from pandas_datareader import data, wb

gist_url_tmpl = "https://gist.githubusercontent.com/plusjune/d5c30a3377d43cd53884/" \
                "raw/e634b4af7616abebb6a1fb41e6c045396de1df20/marcap-%s.csv"

## 2015년 상위 200 종목의 코드와 종목명

In [3]:
this_year = '2015'
df_base = pd.read_csv( gist_url_tmpl % (this_year), thousands=",")
df_base = df_base.head(200)
df_base.head()

,순위,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가총액(백만원),시가총액비중(%),상장주식수(천주),외국인 보유주식수,외국인 지분율(%)
0,1,005930,삼성전자,1295000,"상승 3,000",0.23,207148,268005,190752641,13.04,147299337,72858065,49.46
1,2,005380,현대차,151500,"하락 1,500",-0.98,697842,106227,33371887,2.28,220276479,96759116,43.93
2,3,015760,한국전력,50300,상승 350,0.70,1079479,54193,32290793,2.21,641964077,204358747,31.83
3,4,028260,삼성물산,147500,"하락 1,000",-0.67,278739,41335,27979281,1.91,189690043,18689887,9.85
4,5,005935,삼성전자우,1090000,"상승 5,000",0.46,44463,48372,24888435,1.70,22833427,17276502,75.66


In [3]:
df_base = df_base[['종목코드', '종목명']]
df_base.head()

,종목코드,종목명
0,005930,삼성전자
1,005380,현대차
2,015760,한국전력
3,028260,삼성물산
4,005935,삼성전자우


## 업종 조회하여 채워넣기

In [4]:
import pprint, pickle
import requests
from bs4 import BeautifulSoup


def get_sector(code):
    url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + code
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    sector = ""
    td = soup.find('td', {'class':'cmp-table-cell td0101'})
    dts = td.findAll('dt')
    sector = dts[2].text.split(':')[1].strip()
    return sector


In [5]:
df_sector = df_base.copy()
df_sector['업종'] = ''

In [6]:
for x, row in df_sector.iterrows():
    row['업종'] = get_sector(row['종목코드'])

In [7]:
df_sector.head(20)

,종목코드,종목명,업종
0,005930,삼성전자,전기전자
1,005380,현대차,운수장비
2,015760,한국전력,전기가스
3,028260,삼성물산,유통업
4,005935,삼성전자우,전기전자
5,012330,현대모비스,운수장비
6,090430,아모레퍼시픽,화학
7,000660,SK하이닉스,전기전자
8,051910,LG화학,화학
9,032830,삼성생명,보험업


In [8]:
df_sector.columns = ['ID', 'Entity', 'Group']
df_sector = df_sector.set_index('Entity')
df_sector[['Group', 'ID']].to_csv('marcap_sectors.csv')

In [9]:
!head -10 marcap_sectors.csv

Entity,Group,ID
삼성전자,전기전자,005930
현대차,운수장비,005380
한국전력,전기가스,015760
삼성물산,유통업,028260
삼성전자우,전기전자,005935
현대모비스,운수장비,012330
아모레퍼시픽,화학,090430
SK하이닉스,전기전자,000660
LG화학,화학,051910


## 종목코드를 index로 지정

In [10]:
df_base = df_base[['종목코드', '종목명']]
df_base = df_base.set_index('종목코드')
df_base.head(10)

,종목명
종목코드,
005930,삼성전자
005380,현대차
015760,한국전력
028260,삼성물산
005935,삼성전자우
012330,현대모비스
090430,아모레퍼시픽
000660,SK하이닉스
051910,LG화학


In [12]:
df_base.head(20)

,종목명
종목코드,
005930,삼성전자
005380,현대차
015760,한국전력
028260,삼성물산
005935,삼성전자우
012330,현대모비스
090430,아모레퍼시픽
000660,SK하이닉스
051910,LG화학


In [22]:
def merge_column(df_base, year_start, year_end, merge_column = '현재가'):
    for y in range(year_start, year_end+1):
        dft = pd.read_csv( gist_url_tmpl % (str(y)), thousands=",")
        dft = dft[['종목코드', merge_column]]
        dft = dft.set_index('종목코드')
        dft.columns = [str(y)]
        df_base = df_base.join(dft)
    return df_base

In [23]:
df = merge_column(df_base, 2000, 2015, '현재가')
df = df.set_index('종목명')
df.head(20)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
종목명,,,,,,,,,,,,,,,,
삼성전자,158000,279000,314000,451000,450500,659000,613000,556000,451000,799000,949000,1061000,1522000,1372000,1327000,1295000
현대차,12100,26900,27750,50500,55500,97300,67400,71600,39500,121000,173500,213000,218500,236500,169000,151500
한국전력,23600,21700,18250,21400,26850,37800,42400,39650,29600,34100,30200,26000,30450,34750,42700,50300
삼성물산,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,158000,147500
삼성전자우,69000,113500,150000,245500,298500,491000,480000,428000,258000,525000,649000,656000,852000,1013000,1039000,1090000
현대모비스,4985,18900,21800,64100,65500,92400,85900,87200,63100,171000,284500,290500,288000,293500,236000,250500
아모레퍼시픽,NaN,NaN,NaN,NaN,NaN,NaN,580000,710000,654000,934000,1139000,1081000,1214000,1000000,2220000,414500
SK하이닉스,4025,2420,280,5600,11650,35300,36450,25950,6700,23150,24000,21150,25750,36800,47750,32700
LG화학,NaN,21750,40600,55000,41250,57000,43100,89600,71000,228500,391000,320000,330000,299500,181000,339500


In [24]:
df.to_csv('marcap_price.csv')

In [25]:
!head -10 marcap_price.csv

종목명,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
삼성전자,158000.0,279000.0,314000.0,451000.0,450500.0,659000.0,613000.0,556000.0,451000.0,799000.0,949000.0,1061000.0,1522000.0,1372000.0,1327000.0,1295000
현대차,12100.0,26900.0,27750.0,50500.0,55500.0,97300.0,67400.0,71600.0,39500.0,121000.0,173500.0,213000.0,218500.0,236500.0,169000.0,151500
한국전력,23600.0,21700.0,18250.0,21400.0,26850.0,37800.0,42400.0,39650.0,29600.0,34100.0,30200.0,26000.0,30450.0,34750.0,42700.0,50300
삼성물산,,,,,,,,,,,,,,,158000.0,147500
삼성전자우,69000.0,113500.0,150000.0,245500.0,298500.0,491000.0,480000.0,428000.0,258000.0,525000.0,649000.0,656000.0,852000.0,1013000.0,1039000.0,1090000
현대모비스,4985.0,18900.0,21800.0,64100.0,65500.0,92400.0,85900.0,87200.0,63100.0,171000.0,284500.0,290500.0,288000.0,293500.0,236000.0,250500
아모레퍼시픽,,,,,,,580000.0,710000.0,654000.0,934000.0,1139000.0,1081000.0,1214000.0,1000000.0,2220000.0,414500
SK하이닉스,4025.0,2420.0,280.0,5600.0,11650.0,35300.0,36450.0,25950.0,6

In [26]:
len(df)

200

In [27]:
df = merge_column(df_base, 2000, 2015, '시가총액(백만원)')
df = df.set_index('종목명')
df.to_csv('marcap_cap.csv')
df.head(10)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
종목명,,,,,,,,,,,,,,,,
삼성전자,23895911,42220610,47958513,68034797,66358351,97070263,90294494,81898431,66432001,117692170,139787071,156284597,224189591,202094690,195466220,190752641
현대차,2771973,5893486,6079711,11085684,12133871,21314412,14792026,15742965,8700921,26653454,38217969,46918890,48130411,52095387,37226725,33371887
한국전력,15106381,13890189,11681841,13698159,17204099,24220296,27202471,25438160,18990404,21877459,19375345,16691066,19547806,22308252,27411866,32290793
삼성물산,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21330000,27979281
삼성전자우,1648646,2711904,3584014,5750451,6815778,11211213,10960045,9772707,5891024,11987549,14818894,14978728,19454080,23130262,23723931,24888435
현대모비스,393399,1492702,1856897,5436833,5601509,7918922,7371248,7628121,5526993,16645801,27694329,28278392,28035033,28570424,22973152,24384638
아모레퍼시픽,NaN,NaN,NaN,NaN,NaN,NaN,3390592,4150553,3823185,5460023,6658422,6319363,7096861,5845849,12977785,24231044
SK하이닉스,1973986,2447240,1467192,2484839,5182685,15806375,16731351,11916983,3079339,13650133,14166637,12524429,17874434,26135393,34762113,23805677
LG화학,NaN,1401245,2615658,3543379,2657534,3672229,2776720,6741394,5341953,15142946,25912000,21206752,21869463,19848194,11995069,22499038


In [28]:
df.min()
df.max()

2000     23895911
2001     42220610
2002     47958513
2003     68034797
2004     66358351
2005     97070263
2006     90294494
2007     81898431
2008     66432001
2009    117692170
2010    139787071
2011    156284597
2012    224189591
2013    202094690
2014    195466220
2015    190752641
dtype: float64

In [29]:
df = merge_column(df_base, 2000, 2015, '외국인 지분율(%)')
df = df.set_index('종목명')
df.to_csv('marcap_foreign_ratio.csv')

In [30]:
df.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
종목명,,,,,,,,,,,,,,,,
삼성전자,54.16,59.63,53.90,57.30,54.13,53.83,49.08,46.90,43.07,47.71,50.58,50.40,50.40,49.67,51.81,49.46
현대차,40.98,52.76,47.19,51.25,55.83,45.10,40.79,33.16,26.57,36.40,42.75,42.24,45.85,45.16,43.59,43.93
한국전력,26.14,26.58,24.95,28.93,30.51,29.95,29.28,27.94,24.29,25.46,23.66,23.97,25.77,23.57,29.35,31.83
삼성물산,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.08,9.85
삼성전자우,53.19,64.47,60.67,77.72,80.58,81.87,80.89,80.68,79.72,81.42,82.86,82.22,82.46,82.93,80.18,75.66


In [ ]:
#  End of Document